In [4]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

print("Libraries imported.")

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                            /failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|1_73_pypy|2_73_pypy|4_73_pypy|3_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -> python[version='2.

In [15]:
pip install folium

     |████████████████████████████████| 93 kB 4.0 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import folium

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [12]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501024, -73.9495829.


In [17]:
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

In [18]:
CLIENT_ID = '1YDYHFGGTMYZN5HJMIIHNLDQPDR1M5ABSNAYTLHHDR3WRROP'
CLIENT_SECRET = 'NDBIHT52PENZISYB5H3T2IBFR2DAYFOMXM3ZQOFG5WNPW04Y'
VERSION = '20180604'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1YDYHFGGTMYZN5HJMIIHNLDQPDR1M5ABSNAYTLHHDR3WRROP
CLIENT_SECRET:NDBIHT52PENZISYB5H3T2IBFR2DAYFOMXM3ZQOFG5WNPW04Y


In [19]:
neighborhood_latitude = brooklyn_data.loc[0, 'Latitude']
neighborhood_longitude = brooklyn_data.loc[0, 'Longitude']

neighborhood_name = brooklyn_data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [20]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=1YDYHFGGTMYZN5HJMIIHNLDQPDR1M5ABSNAYTLHHDR3WRROP&client_secret=NDBIHT52PENZISYB5H3T2IBFR2DAYFOMXM3ZQOFG5WNPW04Y&v=20180604&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd69a2ab6ae8d7ec33cf589'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 81,
  'suggestedBounds': {'ne': {'lat': 40.63030106951066,
    'lng': -74.02470273356597},
   'sw': {'lat': 40.62130106051065, 'lng': -74.03653865351028}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b895827f964a5206c2d32e3',
       'name': 'Pilo Arts Day Spa and Salon',
       'location': {'address': '8412 3rd Ave',
        'lat': 40.62474788273414,
        'lng': -74.03059056940135,
        'labeledLatL

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Leo's Casa Calamari,Pizza Place,40.624200,-74.030931
3,Georgian Dream Cafe and Bakery,Caucasian Restaurant,40.625586,-74.030196
4,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
...,...,...,...,...
76,Smart Addiction,Electronics Store,40.623403,-74.030864
77,Fresh Tortillas 2,Mexican Restaurant,40.623180,-74.027984
78,Fort Hamilton Playground,Playground,40.626513,-74.036028
79,Russell Pedersen Playground,Playground,40.626535,-74.036078


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
brooklyn_venues = getNearbyVenues(names = brooklyn_data['Neighborhood'],
                                  latitudes = brooklyn_data['Latitude'],
                                  longitudes = brooklyn_data['Longitude'])

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [27]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2747, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
4,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot


In [28]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,46,46,46,46,46,46
Bay Ridge,81,81,81,81,81,81
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,36,36,36,36,36,36
Bergen Beach,7,7,7,7,7,7
...,...,...,...,...,...,...
Vinegar Hill,29,29,29,29,29,29
Weeksville,16,16,16,16,16,16
Williamsburg,30,30,30,30,30,30


In [29]:
print('There are {} unique categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 291 unique categories.


In [31]:
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
brooklyn_onehot.shape

(2747, 291)

In [33]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bath Beach,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.0,0.021739,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,Bay Ridge,0.012346,0.0,0.000000,0.037037,0.000000,0.0,0.0,0.000000,0.00,...,0.0,0.012346,0.0,0.012346,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,Bedford Stuyvesant,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.035714,0.035714,0.0,0.0
3,Bensonhurst,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,Bergen Beach,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Vinegar Hill,0.000000,0.0,0.000000,0.034483,0.034483,0.0,0.0,0.068966,0.00,...,0.0,0.000000,0.0,0.000000,0.0,0.034483,0.034483,0.034483,0.0,0.0
66,Weeksville,0.000000,0.0,0.000000,0.062500,0.000000,0.0,0.0,0.000000,0.00,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
67,Williamsburg,0.033333,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.033333,0.00,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.033333,0.000000,0.0,0.0
68,Windsor Terrace,0.000000,0.0,0.000000,0.040000,0.040000,0.0,0.0,0.000000,0.04,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.040000,0.0,0.0


In [34]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0  Italian Restaurant  0.04
1          Donut Shop  0.04
2  Chinese Restaurant  0.04
3         Pizza Place  0.04
4            Pharmacy  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.07
1                  Spa  0.06
2          Pizza Place  0.05
3                  Bar  0.04
4  American Restaurant  0.04


----Bedford Stuyvesant----
            venue  freq
0     Coffee Shop  0.11
1            Café  0.07
2     Pizza Place  0.07
3             Bar  0.07
4  Discount Store  0.04


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.11
1  Italian Restaurant  0.08
2         Pizza Place  0.06
3                Park  0.06
4          Donut Shop  0.06


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.29
1  Athletics & Sports  0.14
2      Baseball Field  0.14
3                Park  0.14
4        Hockey Field  0.14


----Boerum Hill----
          venue  freq
0  Dance Studio  0.06

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Fast Food Restaurant,Gas Station,Bubble Tea Shop,Cantonese Restaurant,Italian Restaurant,Chinese Restaurant,Pizza Place,Donut Shop,Dessert Shop
1,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Bar,American Restaurant,Greek Restaurant,Playground,Hookah Bar,Ice Cream Shop,Pharmacy
2,Bedford Stuyvesant,Coffee Shop,Pizza Place,Café,Bar,Deli / Bodega,BBQ Joint,Park,Cocktail Bar,New American Restaurant,Bus Stop
3,Bensonhurst,Chinese Restaurant,Italian Restaurant,Park,Sushi Restaurant,Pizza Place,Ice Cream Shop,Donut Shop,Pet Store,Record Shop,Cosmetics Shop
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Athletics & Sports,Hockey Field,Park,Food & Drink Shop,Food,Food Court,Food Stand


In [38]:
kclusters = 4

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

kmeans.labels_[0:10]

array([1, 3, 3, 1, 3, 3, 1, 1, 1, 3], dtype=int32)

In [39]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,3,Italian Restaurant,Spa,Pizza Place,Bar,American Restaurant,Greek Restaurant,Playground,Hookah Bar,Ice Cream Shop,Pharmacy
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Chinese Restaurant,Italian Restaurant,Park,Sushi Restaurant,Pizza Place,Ice Cream Shop,Donut Shop,Pet Store,Record Shop,Cosmetics Shop
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Pizza Place,Latin American Restaurant,Bank,Bakery,Mexican Restaurant,Mobile Phone Shop,Fried Chicken Joint,Deli / Bodega,Ice Cream Shop,Gym
3,Brooklyn,Greenpoint,40.730201,-73.954241,3,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Record Shop,Yoga Studio,Mexican Restaurant,French Restaurant,Nail Salon,Deli / Bodega
4,Brooklyn,Gravesend,40.595260,-73.973471,1,Lounge,Pizza Place,Bakery,Chinese Restaurant,Italian Restaurant,Gym,Diner,Cosmetics Shop,Furniture / Home Store,Men's Store


In [40]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Women's Store,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food


In [42]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,Chinese Restaurant,Italian Restaurant,Park,Sushi Restaurant,Pizza Place,Ice Cream Shop,Donut Shop,Pet Store,Record Shop,Cosmetics Shop
2,Sunset Park,Pizza Place,Latin American Restaurant,Bank,Bakery,Mexican Restaurant,Mobile Phone Shop,Fried Chicken Joint,Deli / Bodega,Ice Cream Shop,Gym
4,Gravesend,Lounge,Pizza Place,Bakery,Chinese Restaurant,Italian Restaurant,Gym,Diner,Cosmetics Shop,Furniture / Home Store,Men's Store
5,Brighton Beach,Eastern European Restaurant,Restaurant,Russian Restaurant,Mobile Phone Shop,Beach,Sushi Restaurant,Pharmacy,Gourmet Shop,Diner,Taco Place
7,Manhattan Terrace,Ice Cream Shop,Pizza Place,Donut Shop,Chinese Restaurant,Bagel Shop,Burger Joint,Mobile Phone Shop,Organic Grocery,Steakhouse,Grocery Store
8,Flatbush,Coffee Shop,Deli / Bodega,Mexican Restaurant,Caribbean Restaurant,Bank,Pharmacy,Juice Bar,Bagel Shop,Hardware Store,Lounge
9,Crown Heights,Pizza Place,Museum,Café,Bakery,Bagel Shop,Burger Joint,Candy Store,Supermarket,Sushi Restaurant,Salon / Barbershop
10,East Flatbush,Department Store,Liquor Store,Pharmacy,Park,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Food & Drink Shop,Print Shop,Supermarket
11,Kensington,Thai Restaurant,Grocery Store,Ice Cream Shop,Pizza Place,Sandwich Place,Deli / Bodega,Café,Gas Station,Supermarket,Furniture / Home Store
14,Brownsville,Moving Target,Fried Chicken Joint,Restaurant,Chinese Restaurant,Spanish Restaurant,Park,Performing Arts Venue,Trail,Convenience Store,Farmers Market


In [43]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Canarsie,Thai Restaurant,Asian Restaurant,Gym,Grocery Store,Caribbean Restaurant,Food,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop
59,Paerdegat Basin,Asian Restaurant,Child Care Service,Auto Garage,Food,Women's Store,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market


In [44]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Bar,American Restaurant,Greek Restaurant,Playground,Hookah Bar,Ice Cream Shop,Pharmacy
3,Greenpoint,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Record Shop,Yoga Studio,Mexican Restaurant,French Restaurant,Nail Salon,Deli / Bodega
6,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Sandwich Place,Yoga Studio,Diner,Russian Restaurant,Restaurant,Creperie,Pizza Place,Buffet
12,Windsor Terrace,Diner,Park,Plaza,Grocery Store,Deli / Bodega,Bagel Shop,Chinese Restaurant,Café,Butcher,French Restaurant
13,Prospect Heights,Bar,Mexican Restaurant,Thai Restaurant,Bakery,Wine Shop,Cocktail Bar,Gourmet Shop,Southern / Soul Food Restaurant,Ice Cream Shop,Pizza Place
15,Williamsburg,Coffee Shop,Bagel Shop,Yoga Studio,Tapas Restaurant,Steakhouse,Breakfast Spot,Burger Joint,Café,Clothing Store,Diner
16,Bushwick,Bar,Mexican Restaurant,Coffee Shop,Deli / Bodega,Discount Store,Thrift / Vintage Store,Bakery,Vegetarian / Vegan Restaurant,Café,Pizza Place
17,Bedford Stuyvesant,Coffee Shop,Pizza Place,Café,Bar,Deli / Bodega,BBQ Joint,Park,Cocktail Bar,New American Restaurant,Bus Stop
18,Brooklyn Heights,Deli / Bodega,Yoga Studio,Italian Restaurant,Park,Gym,Cosmetics Shop,Pizza Place,Pharmacy,Pet Store,Mexican Restaurant
19,Cobble Hill,Pizza Place,Playground,Coffee Shop,Cocktail Bar,Yoga Studio,Deli / Bodega,Italian Restaurant,Bar,Ice Cream Shop,Middle Eastern Restaurant
